In [117]:
import os

In [118]:
%pwd

'd:\\Repositories\\ml-churn'

In [119]:
os.chdir("d:/Repositories/ml-churn/")

In [120]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/FBrownp/ml-churn.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="FBrownp"
os.environ["MLFLOW_TRACKING_PASSWORD"]="cceabdf6d4f5adc126b6cb03cc7cc4bf568e2591"

In [121]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    model_name: str
    target_column: str 
    mlflow_uri: str
    all_params: dict


In [122]:
from Churn_analysis.constants import *
from Churn_analysis.utils.common import read_yaml, create_directories

In [123]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path  = config.test_data_path,
            model_path = config.model_path,
            model_name = config.model_name_1,
            target_column= schema.target_1,
            mlflow_uri= "https://dagshub.com/FBrownp/ml-churn.mlflow",
            all_params= params
            )
        return model_evaluation_config

In [125]:
import os

from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc


import joblib
import pandas as pd


from Churn_analysis.utils.common import save_json, calculate_metrics

import mlflow
import mlflow.xgboost
from  urllib.parse import urlparse
import numpy as np


In [106]:
class ModelEvaluation():
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def get_model_evaluation_object(self):
        
        test_data_df      = pd.read_csv(self.config.test_data_path)
        
        y_test = test_data_df[self.config.target_column]
        X_test = test_data_df.drop(columns= self.config.target_column)

        model = joblib.load(os.path.join(self.config.model_path,self.config.model_name))

        conf_matrix = confusion_matrix(y_test, model.predict(X_test))

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme



        with mlflow.start_run():

            scores = calculate_metrics(conf_matrix)
            save_json(path= Path(os.path.join(self.config.root_dir,"scores.json")), data = scores)
  
            mlflow.log_params(self.config.all_params)

            for key in scores.keys():
                mlflow.log_metric(key,scores[key])


            if tracking_url_type_store != "file":
                mlflow.xgboost.log_model(model, "Churn_model", registered_model_name="Churn_model")
            else:
                mlflow.xgboost.log_model(model, "Churn_model")



In [107]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.get_model_evaluation_object()

except Exception as e:
    raise e

[2023-12-15 17:06:25,020: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-15 17:06:25,023: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-15 17:06:25,026: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-15 17:06:25,027: INFO: common: created directory at: artifacts]
[2023-12-15 17:06:25,028: INFO: common: created directory at: artifacts/model_evaluation]


NameError: name 'np' is not defined